# A/B Testing Analysis - E-commerce Dataset
This notebook performs hypothesis testing to compare Control vs Treatment conversion rates.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

## Load Dataset

In [ ]:
ab_data = pd.read_csv('ab_test_data.csv')
ab_data.head()

## Formulate Hypothesis
H0: New design ≤ Old design
H1: New design > Old design

In [ ]:
conv_old = ab_data[ab_data['group']=='control']['converted'].sum()
conv_new = ab_data[ab_data['group']=='treatment']['converted'].sum()

n_old = ab_data[ab_data['group']=='control'].shape[0]
n_new = ab_data[ab_data['group']=='treatment'].shape[0]

conv_old, conv_new, n_old, n_new

## Two-Proportion Z-Test

In [ ]:
z_score, p_value = proportions_ztest([conv_new, conv_old], [n_new, n_old], alternative='larger')
print('Z-score:', z_score)
print('P-value:', p_value)

## Confidence Interval Visualization

In [ ]:
ci_old = proportion_confint(conv_old, n_old, alpha=0.05)
ci_new = proportion_confint(conv_new, n_new, alpha=0.05)

plt.figure(figsize=(6,4))
plt.errorbar([0,1], [conv_old/n_old, conv_new/n_new],
             yerr=[[conv_old/n_old-ci_old[0], conv_new/n_new-ci_new[0]],
                   [ci_old[1]-conv_old/n_old, ci_new[1]-conv_new/n_new]],
             fmt='o', capsize=6)
plt.xticks([0,1], ['Control','Treatment'])
plt.ylabel('Conversion Rate')
plt.title('95% Confidence Intervals')
plt.show()

## Chi-Square Test (Device vs Conversion)

In [ ]:
contingency = pd.crosstab(ab_data['device'], ab_data['converted'])
chi2, p, dof, exp = stats.chi2_contingency(contingency)
print('Chi-square p-value:', p)

## T-Test (Session Duration Difference)

In [ ]:
control_duration = ab_data[ab_data['group']=='control']['session_duration']
treatment_duration = ab_data[ab_data['group']=='treatment']['session_duration']

t_stat, p_val = stats.ttest_ind(treatment_duration, control_duration)
print('T-test p-value:', p_val)